In [ ]:
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve
from numpy.linalg import norm
from bending_validation import suppress_stdout

In [ ]:
elastic_rods.set_max_num_tbb_threads(4)
linkage = elastic_rods.RodLinkage('../examples/nonuniform_linkage.obj', 10)
mat = elastic_rods.RodMaterial('+', 2000, 0.3, [0.02, 0.02, 0.002, 0.002])
#mat = elastic_rods.RodMaterial('ellipse', 20000, 0.3, [0.02, 0.002])
linkage.setMaterial(mat)

In [ ]:
view = linkage_vis.LinkageViewer(linkage)
view.setCameraParams(((-0.7239722319967554, -2.953428306702387, -0.5451540221763805),
 (0.028326464489972513, -0.2362002372339648, -0.9712914389307234),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
with suppress_stdout(): elastic_rods.restlen_solve(linkage);

In [ ]:
driver=64
jdo = linkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
elastic_rods.compute_equilibrium(linkage, fixedVars=fixedVars)
view.update()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
#cr = open_linkage(linkage, driver, np.pi/4, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=False);
cr = open_linkage(linkage, driver, np.pi/8, 15, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=True, useTargetAngleConstraint=True);

In [ ]:
from compute_vibrational_modes import *
lambdas, modes = compute_vibrational_modes(linkage, fixedVars[:-1], MassMatrixType.FULL, n=4);

In [ ]:
from mode_viewer import ModeViewer
mview = ModeViewer(linkage, modes, lambdas, width=1024, height=768)
mview.show()

In [ ]:
elastic_rods.linkage_deformation_analysis(linkage_rest, linkage_defo, "linkage_analysis.msh")

In [ ]:
linkage_defo.saveVisualizationGeometry("linkage_geom.msh")